In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
import pandas as pd
from langchain.chat_models import ChatOpenAI
load_dotenv()

In [ ]:
base_path = '/home/john/Proyectos/chatbot/chatbot_beneficios/server'
db = FAISS.load_local(base_path + '/output/embeddings_faiss_index', OpenAIEmbeddings())

In [ ]:
comparativo_sura_df = pd.read_csv(base_path + '/data/comparativo_sura.csv')
comparativo_sura_df = comparativo_sura_df.drop(columns='Entidad')
groupby_servicios = comparativo_sura_df.groupby(['Servicio'])
servicios_dfs = {name_group[0]: group.drop(columns='Servicio') for name_group, group in groupby_servicios}

title = '# Comparativo sura' 
subtitle = '## Servicio {}:'
description = 'Descripción tabla: Coberturas Sura por el servicio {} de medicina prepagada Sura Global y Clásica'

text = title
for servicio, df in servicios_dfs.items():
    text += (
        '\n\n'
        + subtitle.format(servicio)
        + '\n\n'
        + description.format(servicio)
        + '\n\n'
        + servicios_dfs[servicio].to_markdown(index=False)
    )

text = text.replace('|:-', '|--')
print(text)

with open(base_path + '/data/comparativo_sura.md', 'w') as f:
    f.write(text)

In [ ]:
comparativo_salud = pd.read_csv(base_path + '/data/Agente Digital - Comparativo Salud 2.csv')
groupby_servicios = comparativo_salud.groupby(['Servicio'])
servicios_dfs = {name_group[0]: group.drop(columns='Servicio') for name_group, group in groupby_servicios}

title = '# Comparativo entidades de salud' 
subtitle = '## Servicio {}:'
description = 'Descripción tabla: Coberturas de entidades y/o prestadores de salud por {}'

text = title
for servicio, df in servicios_dfs.items():
    text += (
        '\n\n'
        + subtitle.format(servicio)
        + '\n\n'
        + description.format(servicio)
        + '\n\n'
        + servicios_dfs[servicio].to_markdown(index=False)
    )

text = text.replace('|:-', '|--')
print(text)

with open(base_path + '/data/comparativo_salud.md', 'w') as f:
    f.write(text)

In [ ]:
comparativo_salud = pd.read_csv(base_path + '/data/Agente Digital - Comparativo Salud 2.csv')
# comparativo_salud = comparativo_salud.drop(columns='Entidad')
# groupby_servicios = comparativo_salud.groupby(['Servicio'])
# servicios_dfs = {name_group[0]: group.drop(columns='Servicio') for name_group, group in groupby_servicios}

title = '# Comparativo salud' 
# subtitle = '## Servicio {}:'
description = 'Descripción tabla: Comparativo de las opciones (planes) de salud'

text = title + '\n\n' + description
text += '\n\n' + comparativo_salud.to_markdown(index=False)
text = text.replace('|:-', '|--')
print(text)

with open(base_path + '/data/comparativo_salud.md', 'w') as f:
    f.write(text)

In [ ]:
preguntas_frecuentes = pd.read_csv(base_path + '/data/Agente Digital - 1_Preguntas_Respuetas.csv', skiprows=1)
filtro = preguntas_frecuentes['para_chatbot'] == 'si'
preguntas_frecuentes = preguntas_frecuentes[filtro]

title = '# Preguntas frecuentes' 
subtitle = '## Pregunta {}:'

text = title
for i, row in preguntas_frecuentes.iterrows():
    text += (
        '\n\n'
        + subtitle.format(i+1)
        + '\n\n'
        + 'Pregunta: ' + row['Pregunta']
        + '\n'
        + 'Respuesta: ' + row['Respuesta']
    )

print(text)

with open(base_path + '/data/preguntas_respuestas.md', 'w') as f:
    f.write(text)

In [ ]:
docs = db.docstore.__dict__['_dict']

docs = pd.DataFrame([
    {
        'id': id,
        'text': doc.page_content,
        'metadata': doc.metadata,
        'tokens': ChatOpenAI().get_num_tokens(doc.page_content)
    } for id, doc in docs.items()
])

docs

In [ ]:
docs.sort_values(by='tokens', ascending=False).head(10)[['metadata', 'tokens']].to_dict(orient='records')

In [ ]:
# $0.0001 / 1K tokens (openai.com/pricing)
print('número de tokens:', docs.tokens.sum())
print('precio:', round(docs.tokens.sum()/1000*0.0001*4000), 'pesos')